In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [10]:
data = pd.read_csv('D:\College\Semester 6 (Magang)\Model\Anomaly Detection\clean_data.csv')

In [11]:
def preprocess_turbine_data(data):
    """
    Preprocess the turbine data for anomaly detection
    """
    # Convert timestamp and create time features
    data['Timestamp'] = pd.to_datetime(data['Timestamp'], utc=True)

    # Create time-based features
    data['Hour'] = data['Timestamp'].dt.hour
    data['Day'] = data['Timestamp'].dt.day
    data['Month'] = data['Timestamp'].dt.month
    data['Weekday'] = data['Timestamp'].dt.weekday

    # Calculate rolling statistics for each feature
    feature_columns = ['Ngp', 'Npt', 'HPC_ASV_Command', 'HPC_ASV_Position',
                      'HPC_Surge_Margin', 'HPC_ASC_Flow_DP', 'HPC_Suction_Press',
                      'HPC_Discharge_Press']

    # Convert features to numeric
    for col in feature_columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    # Calculate rolling means and standard deviations
    window_size = 24  # 24-hour window
    for col in feature_columns:
        data[f'{col}_rolling_mean'] = data[col].rolling(window=window_size).mean()
        data[f'{col}_rolling_std'] = data[col].rolling(window=window_size).std()

    return data.dropna()

In [12]:
def detect_anomalies(data, contamination=0.02):
    """
    Detect anomalies using Isolation Forest
    """
    # Select features for anomaly detection
    feature_cols = [col for col in data.columns
                   if any(x in col for x in ['_rolling_mean', '_rolling_std'])]

    # Scale the features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(data[feature_cols])

    # Train Isolation Forest
    iso_forest = IsolationForest(
        n_estimators=100,
        contamination=contamination,
        random_state=42
    )

    # Fit and predict
    data['anomaly'] = iso_forest.fit_predict(scaled_features)
    data['anomaly_score'] = iso_forest.score_samples(scaled_features)

    return data, iso_forest

In [13]:
def plot_anomalies(data):
    """
    Create visualizations for the anomaly detection results
    """
    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
    fig.suptitle('Turbine Performance Anomaly Detection Results', fontsize=16)

    # Plot 1: Time series with anomalies
    ax1.plot(data['Timestamp'], data['HPC_ASC_Flow_DP'],
             color='blue', label='HPC_ASC_Flow_DP', alpha=0.5)

    # Highlight anomalies
    anomalies = data[data['anomaly'] == -1]
    ax1.scatter(anomalies['Timestamp'], anomalies['HPC_ASC_Flow_DP'],
                color='red', label='Anomalies')

    ax1.set_title('Time Series with Detected Anomalies')
    ax1.set_xlabel('Timestamp')
    ax1.set_ylabel('HPC_ASC_Flow_DP')
    ax1.legend()

    # Plot 2: Anomaly scores distribution
    sns.histplot(data=data, x='anomaly_score', bins=50, ax=ax2)
    ax2.axvline(x=data[data['anomaly'] == -1]['anomaly_score'].max(),
                color='red', linestyle='--', label='Anomaly Threshold')
    ax2.set_title('Distribution of Anomaly Scores')
    ax2.set_xlabel('Anomaly Score')
    ax2.set_ylabel('Count')
    ax2.legend()

    plt.tight_layout()
    return fig

In [ ]:
# Main execution
def main():
    # Load and preprocess data
    data = pd.read_csv('D:\College\Semester 6 (Magang)\Model\Anomaly Detection\clean_data.csv')  # Replace with your data path
    processed_data = preprocess_turbine_data(data)

    # Detect anomalies
    results, model = detect_anomalies(processed_data)

    # Print summary statistics
    print("\nAnomaly Detection Results:")
    print(f"Total samples: {len(results)}")
    print(f"Number of anomalies: {len(results[results['anomaly'] == -1])}")
    print(f"Anomaly percentage: {(len(results[results['anomaly'] == -1]) / len(results)) * 100:.2f}%")

    # Create and show plots
    fig = plot_anomalies(results)
    plt.show()

    return results, model

In [15]:
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_anomalies(data):
    """
    Create visualizations for the anomaly detection results, including a scatter plot
    of anomaly scores vs. HPC_ASC_Flow_DP.
    """
    # Create figure with subplots
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 15))  # Added one more subplot
    fig.suptitle('Turbine Performance Anomaly Detection Results', fontsize=16)

    # Plot 1: Time series with anomalies
    ax1.plot(data['Timestamp'], data['HPC_ASC_Flow_DP'],
             color='blue', label='HPC_ASC_Flow_DP', alpha=0.5)

    # Highlight anomalies
    anomalies = data[data['anomaly'] == -1]
    ax1.scatter(anomalies['Timestamp'], anomalies['HPC_ASC_Flow_DP'],
                color='red', label='Anomalies')

    ax1.set_title('Time Series with Detected Anomalies')
    ax1.set_xlabel('Timestamp')
    ax1.set_ylabel('HPC_ASC_Flow_DP')
    ax1.legend()

    # Plot 2: Anomaly scores distribution with Gaussian curve
    sns.histplot(data=data, x='anomaly_score', bins=50, ax=ax2, kde=False, stat='density', color='blue', label='Anomaly Scores')

    # Fit a normal distribution to the data
    mu, std = norm.fit(data['anomaly_score'])

    # Plot the fitted Gaussian curve
    xmin, xmax = ax2.get_xlim()
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, mu, std)
    ax2.plot(x, p, 'k', linewidth=2, label=f'Gaussian Fit (μ={mu:.2f}, σ={std:.2f})')

    # Add vertical line for anomaly threshold
    if not anomalies.empty: # Check if anomalies exist before accessing max
        threshold = anomalies['anomaly_score'].max()
        ax2.axvline(x=threshold, color='red', linestyle='--', label='Anomaly Threshold')
    else:
        threshold = None # Or handle it in a way that makes sense in your context

    ax2.set_title('Distribution of Anomaly Scores with Gaussian Fit')
    ax2.set_xlabel('Anomaly Score')
    ax2.set_ylabel('Density')
    ax2.legend()

    # Plot 3: Scatter plot of Anomaly Score vs. HPC_ASC_Flow_DP
    ax3.scatter(data['anomaly_score'], data['HPC_ASC_Flow_DP'], c=data['anomaly'].map({1: 'blue', -1: 'red'}), label='Data Points')
    ax3.set_title('Anomaly Score vs. HPC_ASC_Flow_DP')
    ax3.set_xlabel('Anomaly Score')
    ax3.set_ylabel('HPC_ASC_Flow_DP')
    if threshold is not None:
        ax3.axvline(x=threshold, color='red', linestyle='--', label='Anomaly Threshold') # Add the threshold line here as well
    ax3.legend()

    plt.tight_layout()
    return fig

In [16]:
if __name__ == "__main__":
    results, model = main()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Magang/Dataset/clean_data.csv'